#### https://www.kaggle.com/code/alexpengxiao/preprocessing-model-averaging-by-xgb-lgb-1-39/notebook

preprocessing, model averaging by xgb + lgb

In this notebook, we preprocessed the data and feed the data to gradient boosting tree models, and got 1.39 on public leaderboard.

the workflow is as follows:

##### 1. Data preprocessing. The purpose of data preprocessing is to achieve higher time/space efficiency. What we did includes round, constant features removal, duplicate features removal, insignificant features removal, etc. The key here is to ensure the preprocessing shall not hurt the accuracy.
##### 2. Feature transform. The purpose of feature transform is to help the models to better grasp the information in the data, and fight overfitting. What we did includes dropping features which "live" on different distributions on training/testing set, adding statistical features, adding low-dimensional representation as features.
##### 3. Modeling. We used 2 models: xgboost and lightgbm. We averaged the 2 models for the final prediction.


##### step 1: load train & test data, drop duplicate columns, round the features to NUM_OF_DECIMALS decimals. here NUM_OF_DECIMALS is a experience value which can be tuned.

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import warnings

In [ ]:
warnings.filterwarnings("ignore")
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
test_ID = test['ID']
y_train = train['target']
y_train = np.log1p(y_train)